In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
import os
import zipfile

# Define paths
zip_path = '/content/drive/MyDrive/nature_12K.zip'
unzip_target_path = '/content/drive/MyDrive/nature_12K'

# Unzip only if the target folder does not exist
if not os.path.exists(unzip_target_path):
    print("Unzipping and storing in Google Drive...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/drive/MyDrive/nature_12K')
    print("Unzipping complete and saved in Drive!")
else:
    print("Already unzipped in Drive. Skipping unzip.")

Already unzipped in Drive. Skipping unzip.


In [5]:
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 45.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import WandbLogger
import wandb
import matplotlib.pyplot as plt

In [32]:
# Define the configurable CNN model.
class ConfigurableCNN(pl.LightningModule):
    def __init__(self, config):
        super(ConfigurableCNN, self).__init__()
        self.save_hyperparameters()  # Saves config to checkpoint

        self.config = config

        # Map activation string to function
        activation_map = {
            'ReLU': nn.ReLU,
            'GELU': nn.GELU,
            'SiLU': nn.SiLU,
            'Mish': nn.Mish,
        }
        activation_fn = activation_map.get(config.get('activation', 'ReLU'), nn.ReLU)

        # Build 5 convolution blocks: Conv -> Activation -> (BatchNorm) -> MaxPool -> (Dropout)
        conv_layers = []
        in_channels = 3  # Assuming RGB images
        num_layers = config.get('num_conv_layers', 5)
        filters = config.get('filters', [32, 64, 128, 256, 512])
        kernel_sizes = config.get('kernel_sizes', [3] * num_layers)
        use_batchnorm = config.get('use_batchnorm', False)
        dropout_rate = config.get('dropout_rate', 0.0)

        for i in range(num_layers):
            out_channels = filters[i]
            kernel_size = kernel_sizes[i]
            padding = kernel_size // 2  # To maintain spatial dimensions before pooling
            conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
            block = [conv, activation_fn()]
            if use_batchnorm:
                block.append(nn.BatchNorm2d(out_channels))
            # Append max pooling layer to reduce spatial dimensions by a factor of 2
            block.append(nn.MaxPool2d(2))
            if dropout_rate > 0:
                block.append(nn.Dropout2d(dropout_rate))
            conv_layers.extend(block)
            in_channels = out_channels

        self.conv = nn.Sequential(*conv_layers)

        # After 5 pooling operations, the spatial dimensions of a 128x128 image become:
        # 128 / (2^5) = 4 (assuming the dimensions divide evenly)
        final_spatial = 4
        fc_input_dim = filters[-1] * (final_spatial ** 2)

        # Define the fully connected part: Dense layer and final output layer with 10 classes.
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(fc_input_dim, config.get('dense_neurons', 256)),
            activation_fn(),
            nn.Dropout(config.get('dropout_rate_dense', 0.0)),
            nn.Linear(config.get('dense_neurons', 256), 10)
        )

        # Loss function and learning rate
        self.criterion = nn.CrossEntropyLoss()
        self.lr = config.get('lr', 1e-3)

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [33]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.ToTensor()
])
val_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [34]:
base_dir = "/content/drive/MyDrive/nature_12K/inaturalist_12K"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")


In [35]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
val_dataset = ImageFolder(root=val_dir, transform=val_transforms)

print("Training samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))
print("Classes:", train_dataset.classes)

# Create DataLoaders for training and validation.
batch_size = 32  # Adjustable or swept hyperparameter
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Training samples: 9999
Validation samples: 2000
Classes: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [36]:
# Define default hyperparameter configuration (will be overridden during sweeps if needed).
default_config = {
    "num_conv_layers": 5,
    "filters": [32, 64, 128, 256, 512],
    "kernel_sizes": [3, 3, 3, 3, 3],
    "activation": "ReLU",   # Options: ReLU, GELU, SiLU, Mish
    "use_batchnorm": False,
    "dropout_rate": 0.2,       # For the convolution blocks
    "dense_neurons": 256,
    "dropout_rate_dense": 0.3, # For the dense layer
    "lr": 1e-3,
}

max_epochs = 20

In [37]:
def train_model():
    # Merge default config with sweep-provided config if available.
    config = {**default_config, **wandb.config}

    # Construct a descriptive run name using selected hyperparameters.
    run_name = f"act-{config['activation']}_f-{config['filters'][0]}to{config['filters'][-1]}_lr-{config['lr']}"

    # Set up the wandb logger with the custom run name and unified project name.
    wandb_logger = WandbLogger(project="CNN_inaturalist_12K", name=run_name, config=config)

    model = ConfigurableCNN(config)

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator='gpu',
        precision=16,  # Enable mixed-precision (fp16) training
        logger=wandb_logger,
        log_every_n_steps=10,
    )
    trainer.fit(model, train_loader, val_loader)

In [38]:
# Define wandb Sweep configuration
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'activation': {
            'values': ['ReLU', 'GELU', 'SiLU', 'Mish']
        },
        'filters': {
            'values': [
                [32, 64, 128, 256, 512],
                [32, 32, 64, 64, 128]
            ]
        },
        'use_batchnorm': {
            'values': [True, False]
        },
        'dropout_rate': {
            'values': [0.2, 0.3]
        },
        'dense_neurons': {
            'values': [256, 512]
        },
        'lr': {
            'values': [1e-3, 5e-4]
        },
        'dropout_rate_dense': {
            'values': [0.2, 0.3]
        }
    }
}

In [39]:
# Create the sweep – this returns a unique sweep_id.
sweep_id = wandb.sweep(sweep_config, project="CNN_inaturalist_12K")
print("Sweep ID:", sweep_id)

Create sweep with ID: h86jedde
Sweep URL: https://wandb.ai/mrsagarbiswas-iit-madras/CNN_inaturalist_12K/sweeps/h86jedde
Sweep ID: h86jedde


In [40]:
# Start the sweep agent; adjust count as desired (here count=1 for testing).
wandb.agent(sweep_id, train_model, count=1)

wandb: Agent Starting Run: ufd7t3o4 with config:
wandb: 	activation: Mish
wandb: 	dense_neurons: 256
wandb: 	dropout_rate: 0.2
wandb: 	dropout_rate_dense: 0.3
wandb: 	filters: [32, 32, 64, 64, 128]
wandb: 	lr: 0.0005
wandb: 	use_batchnorm: True
wandb: ERROR Run ufd7t3o4 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-37-3712ddfc2773>", line 3, in train_model
wandb: ERROR     config = {**default_config, **wandb.config}
wandb: ERROR              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/lib/preinit.py", line 27, in __getattr__
wandb: ERROR     raise wandb.Error(f"You must call wandb.init() before {self._name}.{key}")
wandb: ERROR wandb.errors.errors.Error: You must call wandb.init() before wandb.config.keys
wandb: ERROR 
